Были выбраны теги UD, как универсальные для многих языков, поэтому не слишком дробные. https://universaldependencies.org/u/pos/all.html/ На мой взгляд, нет смысла разделять причастия, деепричастия и другие формы глагола, так как прич. и деепр. — продуктивные формы и части парадигмы. А разделять имена собственные и нарицательные, работать с аббревиатурами — важная фича. Она поможет при сложных задачах вроде выделения сущностей и извлечения ключевых выражений.

В тестовом корпусе присутствует много омоформ (семью, что, почти, мыло / мыла, домой и т.д.) Аббревиатуры, сокращения с числительными (3-палубный) и имена собственные.


In [2]:
ud_tags = ['ADJ', 'ADP', 'PUNCT', 'ADV', 'AUX', 'SYM', 'INTJ', 'CCONJ', 'X', 'NOUN', 'DET', 'PROPN', 'NUM', 'VERB', 'PART', 'PRON', 'SCONJ']


### Mystem

In [52]:
from pymystem3 import Mystem
m = Mystem()

mystem_analysis = m.analyze(text)

In [53]:
# перевод тегов

def mystem2ud(pos):
    mystem2ud_dict = {'A': 'ADJ',
                      'ADVPRO': 'ADV',
                      'ANUM': 'NUM',
                      'CONJ': 'CCONJ',
                      'PR': 'ADP',
                      'S': 'NOUN',
                      'V': 'VERB',
                      'APRO': 'PRON',
                      'SPRO': 'PRON', }
    
    if pos in ud_tags:
        return pos
    
    elif pos in mystem2ud_dict:
        return mystem2ud_dict[pos]
    
    else:
        return 'X'

In [54]:
# список из двуместных списков токен – тег
mystem_pos_res = []

for token in mystem_analysis:
    
    # если токен не пробел
    if 'analysis' in token:
        
        # если нет грамм. инф.
        if len(token['analysis']) == 0:
            mystem_pos_res.append([token['text'], 'X'])
        
        else:
            gr = token['analysis'][0]['gr']
            pos = gr.split('=')[0].split(',')[0]
            
            ud_pos = mystem2ud(pos)

            mystem_pos_res.append([token['text'], ud_pos])

### pymorphy

In [55]:
from nltk import word_tokenize

from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [56]:
# перевод тегов

def pymorphy2ud(pos):
    pymorphy2ud_dict = {'ADJF': 'ADJ',
                        'ADJS': 'ADJ',
                        'ADVB': 'ADV',
                        'PREP': 'ADP',
                        'CONJ': 'CCONJ',
                        'COMP': 'ADJ',
                        'NPRO': 'PRON',
                        'NUMR': 'NUM',
                        'PRTS': 'VERB',
                        'PRTF': 'VERB',
                        'PNCT': 'PUNCT',
                        'GRND': 'VERB',
                        'INFN': 'VERB',
                        'PRED': 'ADV',
                        'PRCL': 'PART',
                        'None': 'X'}
    
    if pos in ud_tags:
        return pos
    
    elif pos in pymorphy2ud_dict:
        return pymorphy2ud_dict[pos]
    
    else:
        return 'X'

In [57]:
# список из двуместных списков токен – тег
pymorphy_pos_res = []

words = word_tokenize(text)

for token in words:
    first = morph.parse(token)[0]
    pos = str(first.tag)[0:4]

    ud_pos = pymorphy2ud(pos)
    
    if ud_pos != 'PUNCT':
        pymorphy_pos_res.append([first.word, ud_pos])

### Natasha

In [18]:
pip install natasha

  Using cached natasha-1.4.0-py3-none-any.whl (34.4 MB)
  Using cached ipymarkup-0.9.0-py3-none-any.whl (14 kB)
  Using cached navec-0.10.0-py3-none-any.whl (23 kB)
  Using cached razdel-0.5.0-py3-none-any.whl (21 kB)
  Using cached slovnet-0.5.0-py3-none-any.whl (49 kB)
  Using cached yargy-0.15.0-py3-none-any.whl (41 kB)
Note: you may need to restart the kernel to use updated packages.


In [26]:
import natasha

In [58]:
segmenter = Segmenter()
emb = NewsEmbedding()

morph_vocab = MorphVocab()
morph_tagger = NewsMorphTagger(emb)
doc = Doc(text)

doc.segment(segmenter)
doc.tag_morph(morph_tagger)

In [59]:
# Наташа использует теги UD, перевод не нужен

natasha_pos_res = []

for token in doc.tokens:
    if token.pos != 'PUNCT':
        natasha_pos_res.append([token.text, token.pos])

### Accuracy

In [60]:
import csv

ud_gold_pos = []

with open('corpus.tsv', encoding = 'utf-8') as File:
    reader = csv.reader(File, delimiter='\t')
    for line in reader:
        if line[1] != 'PUNCT':
            ud_gold_pos.append(line)

In [61]:
len(ud_gold_pos), len(natasha_pos_res), len(pymorphy_pos_res), len(natasha_pos_res)

(202, 200, 200, 200)

Алгоритм по-разному разбил некоторые слова с дефисом. Они анализироваться не будут.

In [84]:
def accuracy(set_pos_res, length):
    errors = 0
    for i in range(length):
    
        gold_word = ud_gold_pos[i][0].lower()
        compared_word = set_pos_res[i][0].lower()
    
        gold_tag = ud_gold_pos[i][1]
        compared_tag = set_pos_res[i][1]
    
        if (gold_word == compared_word):
            if (gold_tag != compared_tag):
                errors += 1
                print(gold_word, '\t\tgold ', gold_tag, '\t\tpredicted ', compared_tag)

        else:
            # перескочить через одно слово с дефисом.
            # В списке таких проблемных мест 2: по-литра и ICQ-аккаунт.
            # это кривой код, по-хорошему надо делать через рекурсию, но я не успел.
            # По этому сделаем вид, что код нужен только для этой выборки из 200,
            # где мы точно знаем проблемные места
             
            gold_word = ud_gold_pos[i+1][0]
            compared_word = natasha_pos_res[i][0]
            
            gold_tag = ud_gold_pos[i+1][1]
            compared_tag = set_pos_res[i][1]
            
            if gold_word == compared_word:
                if (gold_tag != compared_tag):
                    errors += 1
                    print(gold_word, '\t\tgold ', gold_tag, '\t\tpredicted ', compared_tag)
            
            else:
                gold_word = ud_gold_pos[i+2][0]
                compared_word = natasha_pos_res[i][0]
                
                gold_tag = ud_gold_pos[i+2][1]
                compared_tag = set_pos_res[i][1]
                
                if gold_word == compared_word:
                    if (gold_tag != compared_tag):
                        errors += 1
                        print(gold_word, '\t\tgold ', gold_tag, '\t\tpredicted ', compared_tag)
                else:
                    print(gold_word, compared_word)
                
                
    return (200 - errors)/200

In [85]:
accuracy(natasha_pos_res, 200)

почти 		gold  PART 		predicted  ADV
почти 		gold  VERB 		predicted  ADV
нлп 		gold  NOUN 		predicted  PROPN
домой 		gold  VERB 		predicted  ADV
и 		gold  PART 		predicted  CCONJ
пропылесось 		gold  VERB 		predicted  NOUN
двадцать 		gold  NUM 		predicted  PROPN
10-й 		gold  ADJ 		predicted  NUM
специализация 		gold  NOUN 		predicted  ADJ
дуло 		gold  VERB 		predicted  NOUN
штирлиц 		gold  PROPN 		predicted  NOUN
напасть 		gold  NOUN 		predicted  VERB
осы 		gold  NOUN 		predicted  VERB
вздумали 		gold  VERB 		predicted  NOUN
не пол-литра
приписываемые 		gold  VERB 		predicted  ADJ
Лазутчик 		gold  NOUN 		predicted  PROPN
фотофиксацию 		gold  NOUN 		predicted  ADJ
расположенная 		gold  VERB 		predicted  ADJ
аккаунта ICQ-аккаунта
Червоненкиса 		gold  NOUN 		predicted  PROPN


0.905

In [86]:
accuracy(pymorphy_pos_res, 200)

семью 		gold  NOUN 		predicted  NUM
что 		gold  PRON 		predicted  CCONJ
почти 		gold  PART 		predicted  ADV
все 		gold  DET 		predicted  ADJ
что 		gold  SCONJ 		predicted  CCONJ
почти 		gold  VERB 		predicted  ADV
нлп 		gold  NOUN 		predicted  X
мыла 		gold  VERB 		predicted  NOUN
домой 		gold  VERB 		predicted  ADV
и 		gold  PART 		predicted  CCONJ
бы 		gold  AUX 		predicted  PART
10-й 		gold  ADJ 		predicted  VERB
хуан 		gold  PROPN 		predicted  NOUN
штирлиц 		gold  PROPN 		predicted  NOUN
штирлиц 		gold  PROPN 		predicted  NOUN
дуло 		gold  NOUN 		predicted  VERB
штирлиц 		gold  PROPN 		predicted  NOUN
лесу шел
ели 		gold  NOUN 		predicted  VERB
когда 		gold  SCONJ 		predicted  CCONJ
напасть 		gold  NOUN 		predicted  VERB
не пол-литра
якобы 		gold  ADV 		predicted  CCONJ
Анатолий 		gold  PROPN 		predicted  NOUN
Вассерман 		gold  PROPN 		predicted  NOUN
Христо 		gold  PROPN 		predicted  NOUN
Грозев 		gold  PROPN 		predicted  NOUN
что 		gold  SCONJ 		predicted  CCONJ
ЦРУ 		gold  PROPN

0.82

In [87]:
accuracy(mystem_pos_res, 200)

почти 		gold  PART 		predicted  ADV
все 		gold  DET 		predicted  PRON
что 		gold  SCONJ 		predicted  CCONJ
почти 		gold  VERB 		predicted  ADV
нлп 		gold  NOUN 		predicted  X
домой 		gold  VERB 		predicted  ADV
и 		gold  PART 		predicted  CCONJ
хорошо 		gold  ADV 		predicted  PART
бы 		gold  AUX 		predicted  PART
Сегодня 10-й
хуан 		gold  PROPN 		predicted  NOUN
спущена 3-палубной
штирлиц 		gold  PROPN 		predicted  NOUN
дуло 		gold  VERB 		predicted  NOUN
штирлиц 		gold  PROPN 		predicted  NOUN
штирлиц 		gold  PROPN 		predicted  NOUN
ели 		gold  NOUN 		predicted  VERB
когда 		gold  SCONJ 		predicted  CCONJ
напасть 		gold  NOUN 		predicted  VERB
не пол-литра
якобы 		gold  ADV 		predicted  PART
Анатолий 		gold  PROPN 		predicted  NOUN
Вассерман 		gold  PROPN 		predicted  NOUN
Христо 		gold  PROPN 		predicted  NOUN
Грозев 		gold  PROPN 		predicted  NOUN
что 		gold  SCONJ 		predicted  CCONJ
ЦРУ 		gold  PROPN 		predicted  NOUN
все 		gold  DET 		predicted  PRON
этой 		gold  DET 		predicted  

0.81

Лучший результат у Natasha.